<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/Parseo_de_Discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs

Mounted at /content/drive
 78% 41.0M/52.3M [00:00<00:00, 51.1MB/s]
100% 52.3M/52.3M [00:00<00:00, 59.9MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [102]:
# Cargamos la base
base = pd.read_csv('/content/Scrap PDFs.csv')

In [215]:
base

,Unnamed: 0.1,link,text,type,Unnamed: 0
0,0,https://www.senado.es/legis10/publicaciones/pd...,JUNTA PREPARATORIA Y SESIÓN CONSTITUTIVACORTES...,PdfReader,NaN
1,1,https://www.senado.es/legis10/publicaciones/pd...,PLENOCORTES GENERALES\nDIARIO DE SESIONES\nSEN...,PdfReader,NaN
2,2,https://www.senado.es/legis10/publicaciones/pd...,PLENO\nCORTES GENERALES\nDIARIO DE SESIONES\nS...,PdfReader,NaN
3,3,https://www.senado.es/legis10/publicaciones/pd...,PLENOCORTES GENERALES\nDIARIO DE SESIONES\nSEN...,PdfReader,NaN
4,4,https://www.senado.es/legis10/publicaciones/pd...,PLENO\nCORTES GENERALES\nDIARIO DE SESIONES\nS...,PdfReader,NaN
...,...,...,...,...,...
417,417,https://www.senado.es/legis14/publicaciones/pd...,Núm. 131 Pág. 75 26 de abril de 2023\nCORTES G...,PdfReader,130.0
418,418,https://www.senado.es/legis14/publicaciones/pd...,Núm. 132 Pág. 1 9 de mayo de 2023\nCORTES GENE...,PdfReader,131.0
419,419,https://www.senado.es/legis14/publicaciones/pd...,Núm. 133 Pág. 61 10 de mayo de 2023\nCORTES GE...,PdfReader,132.0
420,420,https://www.senado.es/legis14/publicaciones/pd...,Núm. 134 Pág. 1 16 de mayo de 2023\nCORTES GEN...,PdfReader,133.0


In [216]:
import pandas as pd

def generar(linea):
  lineas = linea[2].strip().split("\n")

  coincidencias_nombre_cargo = []
  coincidencias_discurso = []
  discurso_actual = ""

  for linea in lineas:
      if linea[0].isupper() and (":" in linea or "(" in linea):
          coincidencias_nombre_cargo.append(linea.split(":")[0].strip())
          coincidencias_discurso.append(discurso_actual)
          discurso_actual = linea.strip()
      else:
          discurso_actual += " " + linea.strip()

  # Agregar la última coincidencia de discurso
  coincidencias_discurso.append(discurso_actual)

  # Verificar si hay la misma cantidad de coincidencias para nombre/cargo y discursos
  if len(coincidencias_nombre_cargo) == len(coincidencias_discurso[1:]):
      # Crear el DataFrame de pandas
      data = {
          "Nombre/Cargo": coincidencias_nombre_cargo,
          "Discurso": coincidencias_discurso[1:]
      }
      df = pd.DataFrame(data)
      df["Discurso"] = df["Discurso"].str.strip()

  else:
      print("Error: no coincide la cantidad de coincidencias para nombre/cargo y discursos.")

  df = df[df['Nombre/Cargo'].str.contains(r'[A-Z]{2,}')]
  df['Discurso'] = df.apply(lambda row: re.sub(re.escape(row['Nombre/Cargo']) + r':\s*', '', row['Discurso']), axis=1)
  df['Cargo'] = df['Nombre/Cargo'].apply(lambda x: ' '.join(re.findall(r'\b[A-Z][A-ZÁÉÍÓÚÜÑ-]+\b', x)))
  df['Nombre'] = df['Nombre/Cargo'].apply(lambda x: re.findall(r'\((.*?)\)', x)[0] if re.findall(r'\((.*?)\)', x) else '')
  df['Link']=linea[1]
  return df

In [204]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo', 'Nombre'])

In [217]:
for row in base.values:
  base2 = pd.concat([base2, generar(row)], axis=0)

<ipython-input-216-b15367f49a5d>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Discurso'] = df.apply(lambda row: re.sub(re.escape(row['Nombre/Cargo']) + r':\s*', '', row['Discurso']), axis=1)
<ipython-input-216-b15367f49a5d>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cargo'] = df['Nombre/Cargo'].apply(lambda x: ' '.join(re.findall(r'\b[A-Z][A-ZÁÉÍÓÚÜÑ-]+\b', x)))
<ipython-input-216-b15367f49a5d>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [218]:
base2=base2.reset_index(drop=True)

In [219]:
base2

,Nombre/Cargo,Discurso,Cargo,Nombre,Link
0,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,Se abre la sesión.,PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,NaN
1,EL señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,En cumplimiento de lo establecido en el aparta...,EL PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,NaN
2,El señor LETRADO MAYOR,"Con su permiso, señor presidente. Real Decreto...",LETRADO MAYOR,,NaN
3,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,"Muchas gracias, señor letrado mayor. FORMACIÓN...",PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,NaN
4,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,"A continuación, de acuerdo con el artículo 3 d...",PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,NaN
...,...,...,...,...,...
350818,La señora VICEPRESIDENTA (Narbona Ruiz),Se vota en sus propios términos.,VICEPRESIDENTA,Narbona Ruiz,r
350819,La señora VICEPRESIDENTA (Narbona Ruiz),Queda rechazada.Núm. 135 Pág. 210 17 de mayo d...,VICEPRESIDENTA,Narbona Ruiz,r
350820,FINANCIACIÓN AUTONÓMICA EN TÉRMINOS DE UNA MAY...,FINANCIACIÓN AUTONÓMICA EN TÉRMINOS DE UNA MAY...,FINANCIACIÓN AUTONÓMICA EN TÉRMINOS DE UNA MAY...,Votación,r
350821,La señora VICEPRESIDENTA (Narbona Ruiz),"Se vota en los términos de la transaccional, c...",VICEPRESIDENTA,Narbona Ruiz,r
